In [ ]:
#| default_exp ctx

# CTX
> Utils for working with MRO CTX data

In [ ]:
#| hide
from nbdev.showdoc import show_doc

In [ ]:
#| export

import warnings
from itertools import repeat
from multiprocessing import Pool
from pathlib import Path

import rasterio
import rioxarray as rxr
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map
from yarl import URL

import hvplot.xarray  # noqa
from fastcore.basics import store_attr
from fastcore.script import call_parse
from planetarypy.config import config
from planetarypy.pds.apps import get_index
from planetarypy.utils import catch_isis_error, file_variations, url_retrieve

try:
    from kalasiris.pysis import (
        ProcessError,
        ctxcal,
        ctxevenodd,
        getkey,
        mroctx2isis,
        spiceinit,
        cam2map,
    )
except KeyError:
    warnings.warn("kalasiris has a problem initializing ISIS")

In [ ]:
#| export

warnings.filterwarnings("ignore", category=rasterio.errors.NotGeoreferencedWarning)
baseurl = URL(config.get_value("mro.ctx.datalevels.edr.url"))

In [ ]:
#| export
storage_root = config.storage_root / "missions/mro/ctx"
cache = dict()

In [ ]:
import pandas as pd

pd.set_option("display.max_columns", 1000)

In [ ]:
#| export
def get_edr_index(refresh=False, check_update=False):
    if 'edrindex' in cache:
        return cache['edrindex']
    else:
        edrindex = get_index("mro.ctx", "edr", refresh=refresh, check_update=check_update)
        edrindex["short_pid"] = edrindex.PRODUCT_ID.map(lambda x: x[:15])
        edrindex["month_col"] = edrindex.PRODUCT_ID.map(lambda x: x[:3])
        edrindex.LINE_SAMPLES = edrindex.LINE_SAMPLES.astype(int)
        cache['edrindex'] = edrindex
        return edrindex

In [ ]:
#| export
class CTXEDR:
    """Manage access to EDR data"""

    root = config.get_value("mro.ctx.datalevels.edr.root") or storage_root / "edr"
    with_pid_folder = config.get_value("mro.ctx.datalevels.edr.with_pid_folder")
    with_volume = config.get_value("mro.ctx.datalevels.edr.with_volume")

    def __init__(
            self,
            pid: str,  # CTX product id (pid)
            root: str = "",  # alternative root folder for EDR data
            with_volume=None,  # does the storage path include the volume folder
            with_pid_folder=None,  # control if stuff is stored inside PID folders
            check_for_index_update:bool=False,  # check if newer index is available.
    ):
        self.pid = pid
        self.root = Path(root) if root else Path(self.root)
        self.with_volume = with_volume if with_volume is not None else self.with_volume
        self.with_pid_folder = (with_pid_folder if with_pid_folder is not None else self.with_pid_folder)
        self.check_for_index_update = check_for_index_update
        self.edrindex = None

    @property
    def pid(self):
        "Return product_id"
        return self._pid

    @pid.setter
    def pid(self, value):
        if len(value) == 15:
            self.edrindex = get_edr_index()
            value = self.edrindex.query(f"short_pid=='{value}'").PRODUCT_ID.iloc[0]
        self._pid = value

    @property
    def short_pid(self):
        return self.pid[:15]

    @property
    def meta(self):
        "get the metadata from the index table"
        edrindex = get_edr_index(check_update=self.check_for_index_update)
        s = edrindex.query("PRODUCT_ID == @self.pid").squeeze()
        s.index = s.index.str.lower()
        return s

    @property
    def volume(self):
        "get the PDS volume number for the current product id"
        return self.meta.volume_id.lower()

    @property
    def source_folder(self):
        """Calculate the source folder based on storage options `with_pid_folder` and `with_volume`."""
        base = self.root
        if self.with_volume:
            base = self.root / self.volume
        if self.with_pid_folder:
            base = base / self.pid
        return base

    @property
    def source_path(self):
        """Combine `source_folder` with `pid` into full path."""
        return self.source_folder / f"{self.pid}.IMG"

    @property
    def url(self):
        "Calculate URL from input dataframe row."
        url = baseurl / self.meta.volume_id.lower() / "data" / (self.pid + ".IMG")
        return url

    def download(self, overwrite=False):  # use `overwrite` to download in all cases.
        "Download and store correctly the EDR data, if not locally available."
        if self.source_path.exists() and not overwrite:
            print("File exists. Use `overwrite=True` to download fresh.")
            return
        self.source_folder.mkdir(parents=True, exist_ok=True)
        url_retrieve(self.url, self.source_path)

    def __str__(self):
        "Show some info about yourself when returned in a REPL (like ipython/jupyter)."
        s = f"PRODUCT_ID: {self.pid}\n"
        s += f"URL: {self.url}\n"
        s += f"source_path: {self.source_path}\n"
        return s

    def __repr__(self):
        return self.__str__()

PRODUCT_IDs can be provided in the shortened form (still unique), which are the first 15 characters of the full PRODUCT_ID:

In [ ]:
pid = "F10_039666_1383"

In [ ]:
edr = CTXEDR(pid)

In [ ]:
show_doc(CTXEDR.pid)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L87){target="_blank" style="float:right; font-size:smaller"}

### CTXEDR.pid

>      CTXEDR.pid ()

Return product_id

In [ ]:
edr.pid

'F10_039666_1383_XN_41S315W'

In [ ]:
edr.short_pid

'F10_039666_1383'

These are the storage configuration settings:

In [ ]:
edr.root

Path('/remote/trove/geo/planet/Mars/CTX/pds')

In [ ]:
edr.with_pid_folder

False

In [ ]:
edr.with_volume

True

In [ ]:
show_doc(CTXEDR.source_folder)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L111){target="_blank" style="float:right; font-size:smaller"}

### CTXEDR.source_folder

>      CTXEDR.source_folder ()

Calculate the source folder based on storage options `with_pid_folder` and `with_volume`.

In [ ]:
edr.source_folder

Path('/remote/trove/geo/planet/Mars/CTX/pds/mrox_2337')

In [ ]:
show_doc(CTXEDR.source_path)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L121){target="_blank" style="float:right; font-size:smaller"}

### CTXEDR.source_path

>      CTXEDR.source_path ()

Combine `source_folder` with `pid` into full path.

In [ ]:
edr.source_path

Path('/remote/trove/geo/planet/Mars/CTX/pds/mrox_2337/F10_039666_1383_XN_41S315W.IMG')

In [ ]:
edr.source_path.exists()

True

In [ ]:
show_doc(CTXEDR.meta)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L98){target="_blank" style="float:right; font-size:smaller"}

### CTXEDR.meta

>      CTXEDR.meta ()

get the metadata from the index table

In [ ]:
edr.meta

volume_id                                                 MROX_2337
file_specification_name         DATA/F10_039666_1383_XN_41S315W.IMG
original_product_id                                4A_04_10C800EF00
product_id                               F10_039666_1383_XN_41S315W
image_time                               2015-01-12 06:36:38.896000
instrument_id                                                   CTX
instrument_mode_id                                             NIFL
line_samples                                                   5056
lines                                                         15360
spatial_summing                                                   1
scaled_pixel_width                                             5.04
pixel_aspect_ratio                                              1.2
emission_angle                                                 1.29
incidence_angle                                                41.1
phase_angle                                     

In [ ]:
show_doc(CTXEDR.url)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L126){target="_blank" style="float:right; font-size:smaller"}

### CTXEDR.url

>      CTXEDR.url ()

Calculate URL from input dataframe row.

In [ ]:
show_doc(CTXEDR.download)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L131){target="_blank" style="float:right; font-size:smaller"}

### CTXEDR.download

>      CTXEDR.download (overwrite=False)

Download and store correctly the EDR data, if not locally available.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| overwrite | bool | False | use `overwrite` to download in all cases. |

In [ ]:
show_doc(CTXEDR.__str__)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L139){target="_blank" style="float:right; font-size:smaller"}

### CTXEDR.__str__

>      CTXEDR.__str__ ()

Show some info about yourself when returned in a REPL (like ipython/jupyter).

In [ ]:
#| export
class CTX:
    """Class to manage dealing with CTX data.

    HAS a CTXEDR attribute as defined above.
    Attributes from CTXEDR are availalbe via __getattr__()
    """
    proc_root = p if (p := Path(config.get_value("mro.ctx.root"))) is True else storage_root / "edr"
    calib_extension = ext if (ext := config.get_value("mro.ctx.calib_extension")) is True else ".cal.cub"
    proc_with_pid_folder = config.get_value("mro.ctx.with_pid_folder")
    proc_with_volume = config.get_value("mro.ctx.with_volume")

    def __init__(
            self,
            id_: str,  # CTX product id
            source_dir: str = "",  # where the raw EDR data is stored, if not coming from plpy
            proc_root: str = "",  # where to store processed, if not plpy
            with_volume=None,  # store with extra volume subfolder?
            with_id_dir=None,  # store with extra product_id subfolder?
    ):
        self.edr = CTXEDR(id_, root=source_dir, with_volume=with_volume)
        store_attr(but="source_dir,proc_root")
        self.proc_root = Path(proc_root) if proc_root else self.proc_root

        (self.cub_name, self.cal_name,
         self.destripe_name, self.map_name) = file_variations(self.edr.source_path.name,
                                               [".cub", self.calib_extension, ".dst.cal.cub", ".lev2.cub"])

        self.is_read = False
        self.is_calib_read = False
        self.checked_destripe = False

    def __getattr__(self, attr):
        return getattr(self.edr, attr)

    @property
    def proc_folder(self) -> Path:
        "the folder for all processed data. could be same as source_dir"
        base = self.proc_root
        if self.proc_with_volume:
            base = base / self.volume
        if self.proc_with_pid_folder:
            base = base / self.pid
        return base

    @property
    def cub_path(self) -> Path:
        "Path to cube after import to ISIS."
        return self.proc_folder / self.cub_name

    @property
    def cal_path(self) -> Path:
        "Path to calibrated cube file. Also destriped files get this name."
        return self.proc_folder / self.cal_name

    @property
    def destripe_path(self) -> Path:
        "One can keep destriped cubes as extra files, but it increases path management complexity."
        return self.proc_folder / self.destripe_name

    @property
    def map_path(self) -> Path:
        return self.proc_folder / self.map_name

    @catch_isis_error
    def isis_import(self) -> None:
        "Import EDR data into ISIS cube."
        self.cub_path.parent.mkdir(exist_ok=True, parents=True)
        mroctx2isis(from_=self.source_path, to=self.cub_path)

    @catch_isis_error
    def spice_init(self, web="yes") -> None:
        "Perform `spiceinit.`"
        spiceinit(from_=self.cub_path, web=web)

    @catch_isis_error
    def calibrate(self) -> None:
        "Do ISIS `ctxcal`."
        ctxcal(from_=self.cub_path, to=self.cal_path)
        self.is_calib_read = False

    @catch_isis_error
    def destripe(self, do_rename=True) -> None:
        "Do destriping via `ctxevenodd` if allowed by summing status."
        if self.spatial_summing != 2:
            ctxevenodd(from_=self.cal_path, to=self.destripe_path)
            if do_rename:
                self.destripe_path.rename(self.cal_path)

    @catch_isis_error
    def map_project(self, mpp=6.25) -> None:
        "Perform map projection."        
        cam2map(from_=self.cal_path, to=self.map_path, pixres='mpp', resolution=mpp)

    @property
    def spatial_summing(self) -> int:
        "Get the spatial summing value from the index file."
        return self.meta["spatial_summing"]

    @property
    def data_quality(self) -> str:
        "Return the index file content for the DATA_QUALITY_DESC flag."
        return self.meta.data_quality_desc

    def calib_pipeline(self, overwrite=False) -> None:
        "Execute the whole ISIS pipeline for CTX EDR data."
        if self.cal_path.exists() and not overwrite:
            return
        pbar = tqdm("isis_import spice_init calibrate destripe".split())
        for name in pbar:
            pbar.set_description(name)
            getattr(self, name)()
        pbar.set_description("Done.")

    @property
    def edr_da(self):
        """Read EDR into xr.DataArray. Drop superfluous band dimension.

        If it was read before, use stored object for speed-up.
        'da' stands for data-array.
        """
        if not self.is_read:
            if not self.source_path.exists():
                # Doing this by hand because rasterio doesn't throw exception when path is missing.
                raise FileNotFoundError("EDR not downloaded yet.")
            self._edr_da = rxr.open_rasterio(self.source_path).sel(band=1, drop=True)
            self._edr_da.name = f"{self.short_pid} EDR"
            self.is_read = True
        return self._edr_da.drop_vars("spatial_ref")

    @property
    def edr_shape(self):
        return self.edr_da.shape

    @property
    def cal_da(self):
        """Read calibrated ISIS cube into xarray.DataArray using rioxarray.

        Drop superfluous `band` dimension.
        If it was read before, use stored object for speed-up.
        'da' stands for data-array.
        """
        if not self.is_calib_read:
            self._cal_da = rxr.open_rasterio(self.cal_path, masked=True).sel(band=1, drop=True)
            self._cal_da.name = f"{self.short_pid} calibrated"
            self.is_calibd_read = True
        return self._cal_da.drop_vars("spatial_ref")

    @property
    def cal_shape(self):
        return self.cal_da.shape

    def plot_da(self, data):
        """Use hvplot to plot the xarray. Used by plot_calibrated to plot the calibrated array."""
        return data.hvplot(
            x="y",
            y="x",
            rasterize=True,
            cmap="gray",
            width=1000,
            height=400,
            title=self.pid[:15],
        )

    def plot_edr(self):
        "Plot EDR xarray using hvplot."
        return self.plot_da(self.edr_da)

    def plot_calibrated(self):
        "Plot the calibrated xarray using hvplot."
        return self.plot_da(self.cal_da)

    def __str__(self):
        "Print out some infos about yourself."
        s = self.edr.__str__()
        try:
            s += f"Shape: {self.edr_da.shape}"
        except FileNotFoundError:
            s += f"Not downloaded yet."
        return s

    def __repr__(self):
        return self.__str__()

In [ ]:
pid = "N05_064260_1638_XI_16S351W"

In [ ]:
ctx = CTX(pid)

The `CTX` class can have a different root storage as the EDR data, e.g. when there's a PDS mirror somewhere locally, where one does not or cannot write to.

Based on that and the same storage options `with_pid_folder` and `with_volume`, we calculate the final `proc_folder`:

In [ ]:
show_doc(CTX.proc_folder)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L185){target="_blank" style="float:right; font-size:smaller"}

### CTX.proc_folder

>      CTX.proc_folder ()

the folder for all processed data. could be same as source_dir

In [ ]:
ctx.proc_folder

Path('/home/ayek72/mnt/slowdata/planetarypy/missions/mro/ctx/edr/mrox_3629/N05_064260_1638_XI_16S351W')

These can be changed at object creation:

In [ ]:
CTX(pid, with_volume=True, with_id_dir=True).source_folder

Path('/remote/trove/geo/planet/Mars/CTX/pds/mrox_3629')

In [ ]:
show_doc(CTX.cal_path)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L200){target="_blank" style="float:right; font-size:smaller"}

### CTX.cal_path

>      CTX.cal_path ()

Path to calibrated cube file. Also destriped files get this name.

In [ ]:
ctx.cal_path

Path('/home/ayek72/mnt/slowdata/planetarypy/missions/mro/ctx/edr/mrox_3629/N05_064260_1638_XI_16S351W/N05_064260_1638_XI_16S351W.cal.cub')

In [ ]:
show_doc(CTX.calib_pipeline)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L253){target="_blank" style="float:right; font-size:smaller"}

### CTX.calib_pipeline

>      CTX.calib_pipeline (overwrite=False)

Execute the whole ISIS pipeline for CTX EDR data.

In [ ]:
ctx.proc_folder

Path('/home/ayek72/mnt/slowdata/planetarypy/missions/mro/ctx/edr/mrox_3629/N05_064260_1638_XI_16S351W')

In [ ]:
ctx.isis_import()

In [ ]:
ctx.spice_init(web="yes")

In [ ]:
ctx.calibrate()

In [ ]:
ctx.destripe()

In [ ]:
# not executing always, as it takes lot of time
# ctx.map_project()

In [ ]:
ctx.map_path

Path('/home/ayek72/mnt/slowdata/planetarypy/missions/mro/ctx/edr/mrox_3629/N05_064260_1638_XI_16S351W/N05_064260_1638_XI_16S351W.lev2.cub')

In [ ]:
ctx.calib_pipeline()

In [ ]:
show_doc(CTX.plot_edr)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L313){target="_blank" style="float:right; font-size:smaller"}

### CTX.plot_edr

>      CTX.plot_edr ()

Plot EDR xarray using hvplot.

In [ ]:
ctx.plot_edr()

:DynamicMap   []
   :Image   [y,x]   (N05_064260_1638 EDR)

:::{.callout-note}
Note the different shape of EDR data and calibrated data. A few SAMPLES are being used for calibration.
:::

In [ ]:
ctx.edr_shape

(15360, 5056)

In [ ]:
ctx.cal_shape

(15360, 5000)

In [ ]:
ctx.plot_calibrated()

:DynamicMap   []
   :Image   [y,x]   (N05_064260_1638 calibrated)

In [ ]:
ctx.cal_da

<xarray.DataArray 'N05_064260_1638 calibrated' (y: 15360, x: 5000)>
[76800000 values with dtype=float32]
Coordinates:
  * x        (x) float64 0.5 1.5 2.5 3.5 ... 4.996e+03 4.998e+03 4.998e+03 5e+03
  * y        (y) float64 0.5 1.5 2.5 3.5 ... 1.536e+04 1.536e+04 1.536e+04
Attributes:
    BANDWIDTH:        0.15
    BANDWIDTH_UNIT:   micrometers
    WAVELENGTH:       0.65
    WAVELENGTH_UNIT:  micrometers
    scale_factor:     1.0
    add_offset:       0.0
    long_name:        BroadBand

## CTXCollection -

In [ ]:
#| export


class CTXCollection:
    """Class with several helpful methods to work with a set of CTX images.

    We identify the images via a list of product_ids.
    Several methods manipulate this list based on the requested constraint.
    """

    @classmethod
    def by_volume(cls, vol_id, **kwargs):
        """Create a CTXCollection from the PDS volume number."""
        if not str(vol_id).startswith("MROX_"):
            vol_id = "MROX_" + str(vol_id)
        query = f"VOLUME_ID=='{vol_id}'"
        edrindex = get_edr_index()
        return cls(edrindex.query(query).PRODUCT_ID.values, edrindex=edrindex, **kwargs)

    @classmethod
    def by_month(cls, month_letters, nth_volume=None, **kwargs):
        """Create a CTXCollection based on the first 3 letters of the product_id (a.k.a. "month")"""
        edrindex = get_edr_index()
        df = edrindex[edrindex.PRODUCT_ID.str.startswith(month_letters)]
        obj = cls(df.PRODUCT_ID.values, **kwargs)
        if nth_volume is not None:
            return cls.by_volume(obj.volumes_in_pids[nth_volume], edrindex=edrindex, **kwargs)
        else:
            return obj

    @classmethod
    def volume_from_pid(cls, pid, **kwargs):
        """Get a CTXCollection of the volume for a given image (product_id)."""
        edrindex = get_edr_index()
        vol = edrindex.query(f"PRODUCT_ID=='{pid}'").VOLUME_ID.iat[0]
        return cls.by_volume(vol, **kwargs)

    def __init__(self, product_ids, full_width=False, filter_error=False, edrindex=None):
        self.product_ids = product_ids
        self.full_width = full_width  # i.e. LINE_SAMPLES==5056
        self.filter_error = filter_error
        self.edrindex = get_edr_index() if edrindex is None else edrindex

    @property
    def pids(self):
        "Alias on product_id"
        return self.product_ids

    @property
    def product_ids(self):
        new_pids = self._product_ids
        ind = self.edrindex[self.edrindex.PRODUCT_ID.isin(new_pids)]
        queries = []
        if self.full_width:
            queries.append('LINE_SAMPLES == 5056')
            # new_pids = [pid for pid in new_pids if CTX(pid).meta.line_samples == 5056]
        if self.filter_error:
            queries.append("DATA_QUALITY_DESC != 'ERROR'")
            # new_pids = [pid for pid in new_pids if CTX(pid).data_quality != 'ERROR']
        if queries:
            return ind.query(" and ".join(queries)).PRODUCT_ID.values
        else:
            return ind.PRODUCT_ID.values

    @product_ids.setter
    def product_ids(self, val):
        self._product_ids = val

    def get_urls(self):
        """Get URLs for list of product_ids.

        Returns
        -------
        List[yarl.URL]
            List of URL objects with the respective PDS URL for download.
        """
        urls = []
        for p_id in self.product_ids:
            ctx = CTXEDR(p_id)
            urls.append(ctx.url)
        self.urls = urls
        return urls

    def _do_download(self, args):
        pid, overwrite = args
        ctx = CTX(pid)
        ctx.download(overwrite=overwrite)

    def download_collection(self, overwrite=False):
        "download the images in parallel using tqdm wrapper around concurrent.future"
        print("Downloading collection...")
        args = zip(self.product_ids, repeat(overwrite))
        r = process_map(self._do_download, args, max_workers=6)

    def _do_calib(self, args):
        pid, overwrite = args
        ctx = CTX(pid)
        ctx.calib_pipeline(overwrite=overwrite)

    def calibrate_collection(self, overwrite=False):
        "Calibrate all images in collection using tqdm wrapper around concurrent.future"
        print("Launching parallel calibration...")
        args = zip(self.product_ids, repeat(overwrite))
        process_map(self._do_calib, args, max_workers=6)

    def edr_exist_check(self):
        "Check if all source_paths exists, i.e. all EDR images are available."
        return [(p_id, CTX(p_id).source_path.exists()) for p_id in self.product_ids]

    def calib_exist_check(self):
        "Check if all cal_paths exist. (i.e. all calibrated ISIS cubes are available."
        return [(p_id, CTX(p_id).cal_path.exists()) for p_id in self.product_ids]

    def only_full_width(self):
        "Constrain the list of product_ids to those that have full width (i.e. line_samples == 5056)"

    def get_ctx_n(self, n):
        "Get CTX object for n-th product_id"
        return CTX(self.product_ids[n])

    def get_pid_n(self, n):
        "Get pid for n-th entry in product_ids."
        return self.product_ids[n]

    @property
    def n_items(self):
        "Return length of product_ids list."
        return len(self.pids)

    @property
    def meta(self):
        "Return the index file filtered for the given product_ids."
        return self.edrindex[self.edrindex.PRODUCT_ID.isin(self.pids)]

    @property
    def image_times(self):
        "Return the image observation times."
        return self.meta.IMAGE_TIME

    def get_corrupted(self):
        "Return the product_ids where the PDS index file has an 'ERROR' flag for the `DATA_QUALITY_DESC` field."
        return [pid for pid in self.pids if CTX(pid).data_quality == "ERROR"]

    def filter_error(self):
        "Filter the product_ids for the error flag from the PDS index."
        self.product_ids = [pid for pid in self.pids if CTX(pid).data_quality != "ERROR"]

    @property
    def volumes_in_pids(self):
        return self.edrindex[self.edrindex.PRODUCT_ID.isin(self.product_ids)].VOLUME_ID.unique()

    @property
    def count_per_volume(self):
        g = self.edrindex.groupby("VOLUME_ID")
        return g.size()[self.volumes_in_pids]

    def sample(self, n):
        "Return random sample of product_ids, size `n`."
        return list(pd.Series(self.product_ids).sample(n))

    def __str__(self):
        s = f"# of product IDs: {self.n_items}\n"
        s += "Volumes contained in list of product_ids:\n"
        s += f"{self.volumes_in_pids}\n"
        return s

    def __repr__(self):
        return self.__str__()

The `CTXCollection` class offers a few class methods for a wider range of finding CTX product_ids from the index file:

In [ ]:
show_doc(CTXCollection.by_volume)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L342){target="_blank" style="float:right; font-size:smaller"}

### CTXCollection.by_volume

>      CTXCollection.by_volume (vol_id, **kwargs)

Create a CTXCollection from the PDS volume number.

In [ ]:
CTXCollection.by_volume(4114).n_items

30

In [ ]:
CTXCollection.by_volume(4114, full_width=True).n_items

19

In [ ]:
CTXCollection.by_volume(4114, full_width=True, filter_error=True).n_items

14

In [ ]:
CTXCollection.by_volume(4114, full_width=False, filter_error=True).n_items

23

In [ ]:
show_doc(CTXCollection.by_month)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L351){target="_blank" style="float:right; font-size:smaller"}

### CTXCollection.by_month

>      CTXCollection.by_month (month_letters, nth_volume=None, **kwargs)

Create a CTXCollection based on the first 3 letters of the product_id (a.k.a. "month")

In [ ]:
CTXCollection.by_month("J18", filter_error=True, full_width=True).n_items

287

In [ ]:
show_doc(CTXCollection.volume_from_pid)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L362){target="_blank" style="float:right; font-size:smaller"}

### CTXCollection.volume_from_pid

>      CTXCollection.volume_from_pid (pid, **kwargs)

Get a CTXCollection of the volume for a given image (product_id).

We define an example list of `product_id`s:

In [ ]:
ids = get_edr_index().sample(3, random_state=41).PRODUCT_ID
ids

42799    G18_025164_1658_XN_14S053W
25569    B17_016245_1930_XN_13N274W
13969    P20_008970_2641_XN_84N049W
Name: PRODUCT_ID, dtype: string

In [ ]:
CTXCollection.volume_from_pid(ids.values[0]).n_items  # getting the whole volume here

35

In [ ]:
coll = CTXCollection(ids)

In [ ]:
coll.edr_exist_check()

[('P20_008970_2641_XN_84N049W', True),
 ('B17_016245_1930_XN_13N274W', True),
 ('G18_025164_1658_XN_14S053W', True)]

In [ ]:
coll.get_urls()

[URL('https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_0615/data/P20_008970_2641_XN_84N049W.IMG'),
 URL('https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_0932/data/B17_016245_1930_XN_13N274W.IMG'),
 URL('https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_1452/data/G18_025164_1658_XN_14S053W.IMG')]

The next command launches a parallel download:

In [ ]:
coll.download_collection(overwrite=False)

0it [00:00, ?it/s]

File exists. Use `overwrite=True` to download fresh.
File exists. Use `overwrite=True` to download fresh.
File exists. Use `overwrite=True` to download fresh.


This is performing the ISIS import and calibration in parallel:

In [ ]:
coll.calibrate_collection()

Launching parallel calibration...


0it [00:00, ?it/s]

In [ ]:
coll.calib_exist_check()

[('P20_008970_2641_XN_84N049W', True),
 ('B17_016245_1930_XN_13N274W', True),
 ('G18_025164_1658_XN_14S053W', True)]

In [ ]:
coll = CTXCollection.by_volume(4114)

In [ ]:
coll.product_ids

<StringArray>
['N20_069979_1676_XI_12S177W', 'N20_069980_1676_XI_12S205W',
 'N20_069981_1919_XI_11N234W', 'N20_069982_1380_XI_42S255W',
 'N20_069982_1820_XI_02N261W', 'N20_069982_2287_XN_48N269W',
 'N20_069983_1442_XI_35S283W', 'N20_069984_1686_XI_11S313W',
 'N20_069984_2097_XI_29N319W', 'N20_069985_2064_XI_26N345W',
 'N20_069986_2025_XI_22N012W', 'N20_069987_2243_XN_44N042W',
 'N20_069991_1451_XI_34S142W', 'N20_069991_1940_XI_14N149W',
 'N20_069992_1761_XI_03S173W', 'N20_069993_1724_XI_07S200W',
 'N20_069994_1753_XI_04S227W', 'N20_069995_1633_XI_16S254W',
 'N20_069995_2028_XI_22N258W', 'N20_069996_2085_XN_28N285W',
 'N20_069997_1479_XI_32S306W', 'N20_069999_1558_XI_24S003W',
 'N20_070004_1931_XI_13N144W', 'N20_070006_1430_XI_37S191W',
 'N20_070007_1793_XI_00S223W', 'N20_070009_2018_XN_21N282W',
 'N20_070010_1466_XI_33S301W', 'N20_070011_1507_XI_29S328W',
 'N20_070011_2252_XI_45N338W', 'N20_070012_1824_XN_02N001W']
Length: 30, dtype: string

In [ ]:
show_doc(CTXCollection.get_corrupted)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L470){target="_blank" style="float:right; font-size:smaller"}

### CTXCollection.get_corrupted

>      CTXCollection.get_corrupted ()

Return the product_ids where the PDS index file has an 'ERROR' flag for the `DATA_QUALITY_DESC` field.

In [ ]:
coll.get_corrupted()

['N20_069991_1451_XI_34S142W',
 'N20_069992_1761_XI_03S173W',
 'N20_069993_1724_XI_07S200W',
 'N20_069994_1753_XI_04S227W',
 'N20_069997_1479_XI_32S306W',
 'N20_070010_1466_XI_33S301W',
 'N20_070012_1824_XN_02N001W']

In [ ]:
show_doc(CTXCollection.n_items)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L456){target="_blank" style="float:right; font-size:smaller"}

### CTXCollection.n_items

>      CTXCollection.n_items ()

Return length of product_ids list.

In [ ]:
coll.n_items

30

In [ ]:
show_doc(CTXCollection.sample)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L487){target="_blank" style="float:right; font-size:smaller"}

### CTXCollection.sample

>      CTXCollection.sample (n)

Return random sample of product_ids, size `n`.

In [ ]:
coll.sample(4)

['N20_069986_2025_XI_22N012W',
 'N20_070006_1430_XI_37S191W',
 'N20_070012_1824_XN_02N001W',
 'N20_069999_1558_XI_24S003W']

In [ ]:
show_doc(CTXCollection.meta)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L461){target="_blank" style="float:right; font-size:smaller"}

### CTXCollection.meta

>      CTXCollection.meta ()

Return the index file filtered for the given product_ids.

In [ ]:
coll.meta.head()

,VOLUME_ID,FILE_SPECIFICATION_NAME,ORIGINAL_PRODUCT_ID,PRODUCT_ID,IMAGE_TIME,INSTRUMENT_ID,INSTRUMENT_MODE_ID,LINE_SAMPLES,LINES,SPATIAL_SUMMING,SCALED_PIXEL_WIDTH,PIXEL_ASPECT_RATIO,EMISSION_ANGLE,INCIDENCE_ANGLE,PHASE_ANGLE,CENTER_LONGITUDE,CENTER_LATITUDE,UPPER_LEFT_LONGITUDE,UPPER_LEFT_LATITUDE,UPPER_RIGHT_LONGITUDE,UPPER_RIGHT_LATITUDE,LOWER_LEFT_LONGITUDE,LOWER_LEFT_LATITUDE,LOWER_RIGHT_LONGITUDE,LOWER_RIGHT_LATITUDE,MISSION_PHASE_NAME,TARGET_NAME,SPACECRAFT_CLOCK_START_COUNT,FOCAL_PLANE_TEMPERATURE,LINE_EXPOSURE_DURATION,OFFSET_MODE_ID,SAMPLE_FIRST_PIXEL,SCALED_IMAGE_WIDTH,SCALED_IMAGE_HEIGHT,SPACECRAFT_ALTITUDE,TARGET_CENTER_DISTANCE,SLANT_DISTANCE,USAGE_NOTE,NORTH_AZIMUTH,SUB_SOLAR_AZIMUTH,SUB_SOLAR_LONGITUDE,SUB_SOLAR_LATITUDE,SUB_SPACECRAFT_LONGITUDE,SUB_SPACECRAFT_LATITUDE,SOLAR_DISTANCE,SOLAR_LONGITUDE,LOCAL_TIME,IMAGE_SKEW_ANGLE,RATIONALE_DESC,DATA_QUALITY_DESC,ORBIT_NUMBER,short_pid,month_col
127546,MROX_4114,DATA/N20_069979_1676_XI_12S177W.IMG,4A_04_1165000100,N20_069979_1676_XI_12S177W,2021-07-01 02:22:53.651,CTX,ITL,5056,43008,1,5.3,1.14,5.3,65.18,61.08,177.4,-12.53,177.37,-14.74,176.91,-14.69,177.89,-10.37,177.43,-10.32,ESP,MARS,1309573428:228,291.3,1.877,196/188/183,0,26.53,260.74,262.71,3657.97,263.76,N,276.79,219.82,233.3,23.12,177.78,-12.56,249116269.6,65.97,15.72,90.2,Northern Terra Sirenum,OK,69979,N20_069979_1676,N20
127547,MROX_4114,DATA/N20_069980_1676_XI_12S205W.IMG,4A_04_1165000200,N20_069980_1676_XI_12S205W,2021-07-01 04:15:32.600,CTX,ITL,5056,18432,1,5.26,1.15,2.48,64.99,63.03,204.93,-12.5,205.04,-13.46,204.59,-13.41,205.26,-11.59,204.81,-11.53,ESP,MARS,1309580187:215,290.9,1.877,196/188/183,0,26.34,111.73,262.54,3657.8,262.77,N,276.75,219.87,260.62,23.13,205.11,-12.51,249117636.2,66.0,15.71,90.1,Valleys in Terra Cimmeria,OK,69980,N20_069980_1676,N20
127548,MROX_4114,DATA/N20_069981_1919_XI_11N234W.IMG,4A_04_1165000300,N20_069981_1919_XI_11N234W,2021-07-01 06:14:46.674,CTX,ITL,5056,52224,1,5.62,1.07,8.13,53.43,46.2,234.64,11.91,234.58,9.25,234.1,9.3,235.2,14.52,234.71,14.57,ESP,MARS,1309587341:234,291.1,1.877,196/188/183,0,28.12,314.36,275.31,3670.67,277.89,N,276.65,206.74,289.76,23.14,235.27,11.85,249118514.0,66.04,15.66,90.0,Nepenthes Planum region,OK,69981,N20_069981_1919,N20
127549,MROX_4114,DATA/N20_069982_1380_XI_42S255W.IMG,4A_04_1165000400,N20_069982_1380_XI_42S255W,2021-07-01 07:50:56.614,CTX,ITL,2528,7168,2,10.25,1.19,4.87,83.98,80.5,255.35,-42.07,255.52,-42.83,254.94,-42.77,255.75,-41.37,255.18,-41.32,ESP,MARS,1309593111:219,291.2,1.884,196/188/183,0,25.65,87.09,254.99,3642.26,255.85,N,276.84,225.48,313.02,23.14,255.8,-42.1,249121323.6,66.07,15.84,90.1,Apron in the Hellas Montes region,OK,69982,N20_069982_1380,N20
127550,MROX_4114,DATA/N20_069982_1820_XI_02N261W.IMG,4A_04_1165000500,N20_069982_1820_XI_02N261W,2021-07-01 08:03:55.615,CTX,ITL,5056,52224,1,5.42,1.11,5.11,57.47,53.17,261.02,2.02,260.94,-0.66,260.49,-0.6,261.57,4.63,261.11,4.69,ESP,MARS,1309593890:219,291.0,1.877,196/188/183,0,27.13,315.79,268.72,3664.89,269.71,N,276.84,213.19,316.33,23.14,261.4,1.98,249120029.2,66.07,15.67,90.1,Tyrrhena Terra,OK,69982,N20_069982_1820,N20


In [ ]:
show_doc(CTXCollection.image_times)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L466){target="_blank" style="float:right; font-size:smaller"}

### CTXCollection.image_times

>      CTXCollection.image_times ()

Return the image observation times.

In [ ]:
coll.image_times

127546   2021-07-01 02:22:53.651
127547   2021-07-01 04:15:32.600
127548   2021-07-01 06:14:46.674
127549   2021-07-01 07:50:56.614
127550   2021-07-01 08:03:55.615
127551   2021-07-01 08:19:18.963
127552   2021-07-01 09:44:54.631
127553   2021-07-01 11:44:29.607
127554   2021-07-01 11:57:15.646
127555   2021-07-01 13:48:19.595
127556   2021-07-01 15:39:04.638
127557   2021-07-01 17:38:53.212
127558   2021-07-02 00:42:16.641
127559   2021-07-02 00:57:36.688
127560   2021-07-02 02:44:15.621
127561   2021-07-02 04:35:30.668
127562   2021-07-02 06:28:38.621
127563   2021-07-02 08:16:50.660
127564   2021-07-02 08:29:27.601
127565   2021-07-02 10:23:41.558
127566   2021-07-02 11:56:54.648
127567   2021-07-02 15:43:37.659
127568   2021-07-03 01:16:03.651
127569   2021-07-03 04:45:28.607
127570   2021-07-03 06:48:35.596
127571   2021-07-03 10:40:22.119
127572   2021-07-03 12:14:51.615
127573   2021-07-03 14:08:48.673
127574   2021-07-03 14:31:39.582
127575   2021-07-03 16:10:56.454
Name: IMAG

Also cool: `pandas` can do time-average:

In [ ]:
coll.image_times.mean()

Timestamp('2021-07-02 05:45:11.936933376')

In [ ]:
show_doc(CTXCollection.get_ctx_n)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L447){target="_blank" style="float:right; font-size:smaller"}

### CTXCollection.get_ctx_n

>      CTXCollection.get_ctx_n (n)

Get CTX object for n-th product_id

In [ ]:
coll.get_ctx_n(2)

PRODUCT_ID: N20_069981_1919_XI_11N234W
URL: https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_4114/data/N20_069981_1919_XI_11N234W.IMG
source_path: /remote/trove/geo/planet/Mars/CTX/pds/mrox_4114/N20_069981_1919_XI_11N234W.IMG
Shape: (52224, 5056)

In [ ]:
show_doc(CTXCollection.get_pid_n)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L451){target="_blank" style="float:right; font-size:smaller"}

### CTXCollection.get_pid_n

>      CTXCollection.get_pid_n (n)

Get pid for n-th entry in product_ids.

In [ ]:
coll.get_pid_n(2)

'N20_069981_1919_XI_11N234W'

In [ ]:
coll = CTXCollection.by_month("N21")

In [ ]:
len(coll.product_ids)

342

In [ ]:
coll

# of product IDs: 342
Volumes contained in list of product_ids:
<StringArray>
['MROX_4126', 'MROX_4127', 'MROX_4128', 'MROX_4129', 'MROX_4130', 'MROX_4131',
 'MROX_4132', 'MROX_4133']
Length: 8, dtype: string

## Command line interfaces

In [ ]:
#| export
@call_parse
def ctx_calib(
        pid: str,  # CTX product_id
        source: str = "",  # path to where EDRs are stored if not from plpy
        proc_root: str = "",  # path to where processed data is to be stored
        overwrite: bool = False,  # overwrite processed data
):
    ctx = CTX(pid, source_dir=source, proc_root=proc_root)
    ctx.calib_pipeline(overwrite=overwrite)
    print("Produced\n", ctx.cal_path)

In [ ]:
ctx_calib(pid, overwrite=True)

  0%|          | 0/4 [00:00<?, ?it/s]

Produced
 /home/ayek72/mnt/slowdata/planetarypy/missions/mro/ctx/edr/mrox_3629/N05_064260_1638_XI_16S351W/N05_064260_1638_XI_16S351W.cal.cub


In [ ]:
from nbdev import nbdev_export

nbdev_export()